# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv("../WeatherPy/city_weather.csv")
city_weather_df

,city,country,date,lat,lng,temp,max temp,humidity,cloudiness,wind_speed_mph
0,mataura,NZ,1620175024,-46.1927,168.8643,57.99,57.99,52,9,5.01
1,msowero,TZ,1620175025,-6.5333,37.2000,69.33,69.33,91,100,4.36
2,hilo,US,1620174797,19.7297,-155.0900,75.78,82.40,57,20,10.36
3,saskylakh,RU,1620175026,71.9167,114.0833,29.82,29.82,93,100,8.70
4,nikolskoye,RU,1620175026,59.7035,30.7861,35.20,35.60,93,75,4.47
...,...,...,...,...,...,...,...,...,...,...
593,pacific grove,US,1620175362,36.6177,-121.9166,58.32,59.00,83,31,7.85
594,samarai,PG,1620175412,-10.6167,150.6667,82.24,82.24,72,16,12.08
595,lompoc,US,1620175412,34.6391,-120.4579,66.87,72.00,59,2,12.66
596,mercedes,AR,1620175413,-34.6515,-59.4307,51.66,52.00,78,100,1.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
gmaps

<module 'gmaps' from 'C:\\Users\\megan\\Anaconda3\\lib\\site-packages\\gmaps\\__init__.py'>

In [5]:
locations = city_weather_df[["lat", "lng"]]
weight = city_weather_df["humidity"]


In [49]:
fig = gmaps.figure(zoom_level=1.9, center = (0.0,0.0))

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
nice_weather_df = city_weather_df.loc[(city_weather_df['max temp']<80) & (city_weather_df['max temp']>70) & (city_weather_df['wind_speed_mph']<4.5) & (city_weather_df['cloudiness']==0)]
nice_weather_df

,city,country,date,lat,lng,temp,max temp,humidity,cloudiness,wind_speed_mph
50,cabedelo,BR,1620175054,-6.9811,-34.8339,73.40,73.40,100,0,3.44
91,pitimbu,BR,1620175078,-7.4706,-34.8086,73.40,73.40,100,0,3.44
233,touros,BR,1620175167,-5.1989,-35.4608,75.20,75.20,94,0,2.30
259,penha,BR,1620175185,-26.7694,-48.6458,71.60,71.60,94,0,2.30
309,manicore,BR,1620175220,-5.8092,-61.3003,77.00,77.00,94,0,2.30
491,ibra,OM,1620175343,22.6906,58.5334,74.46,74.46,78,0,2.46
558,rio de janeiro,BR,1620175197,-22.9028,-43.2075,74.28,75.20,88,0,2.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = nice_weather_df
hotel_df["hotel name"]=""
hotel_df


<ipython-input-14-4bd7ff0e91a5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["hotel name"]=""


,city,country,date,lat,lng,temp,max temp,humidity,cloudiness,wind_speed_mph,hotel name
50,cabedelo,BR,1620175054,-6.9811,-34.8339,73.40,73.40,100,0,3.44,
91,pitimbu,BR,1620175078,-7.4706,-34.8086,73.40,73.40,100,0,3.44,
233,touros,BR,1620175167,-5.1989,-35.4608,75.20,75.20,94,0,2.30,
259,penha,BR,1620175185,-26.7694,-48.6458,71.60,71.60,94,0,2.30,
309,manicore,BR,1620175220,-5.8092,-61.3003,77.00,77.00,94,0,2.30,
491,ibra,OM,1620175343,22.6906,58.5334,74.46,74.46,78,0,2.46,
558,rio de janeiro,BR,1620175197,-22.9028,-43.2075,74.28,75.20,88,0,2.30,


In [15]:
#Google Places nearby API Call
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"


cities = hotel_df['city'].to_list()
lats = hotel_df['lat'].to_list()
lngs= hotel_df['lng'].to_list()


# rio=[-22.9028,-43.2075]
hotels = []



for lat,lng,city in zip(lats,lngs,cities):
    print(lat,lng)
    query_parameters =f"key={g_key}&location={lat},{lng}&radius=5000&keyword=hotel"
    response=requests.get(url+query_parameters).json()
    try:
        hotels.append(response['results'][0]['name'])
    except: 
        hotels.append(None)
        print(f"Could not find hotel within 5000 meters of {city}")

-6.9811 -34.8339
-7.4706 -34.8086
-5.1989 -35.4608
-26.7694 -48.6458
-5.8092 -61.3003
22.6906 58.5334
-22.9028 -43.2075


In [16]:
hotel_df['hotel name'] = hotels
hotel_df

<ipython-input-16-bac39379d63c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['hotel name'] = hotels


,city,country,date,lat,lng,temp,max temp,humidity,cloudiness,wind_speed_mph,hotel name
50,cabedelo,BR,1620175054,-6.9811,-34.8339,73.40,73.40,100,0,3.44,Hotel Almagre
91,pitimbu,BR,1620175078,-7.4706,-34.8086,73.40,73.40,100,0,3.44,Asenza Beach Resort
233,touros,BR,1620175167,-5.1989,-35.4608,75.20,75.20,94,0,2.30,Hotel Vila Gale Touros
259,penha,BR,1620175185,-26.7694,-48.6458,71.60,71.60,94,0,2.30,Pousada Pedra da Ilha
309,manicore,BR,1620175220,-5.8092,-61.3003,77.00,77.00,94,0,2.30,Hotel Durval
491,ibra,OM,1620175343,22.6906,58.5334,74.46,74.46,78,0,2.46,Golden Rays hotel
558,rio de janeiro,BR,1620175197,-22.9028,-43.2075,74.28,75.20,88,0,2.30,Mama Ruisa


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["lat", "lng"]]

In [43]:
# Add marker layer ontop of heat map
fig = gmaps.figure(zoom_level=3, center = (5.0,-15.0))
markers = gmaps.marker_layer(marker_locations,info_box_content =hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))